In [2]:
import pandas as pd

# Загрузка файла CSV
file_path = "/content/drive/MyDrive/new_dataframe22.csv"
df = pd.read_csv(file_path)
df.head()


,Unnamed: 0,user_id,hour,os,order_class,surge,app_opened,price_seen,order_made,ride_completed,user_cancelled,city_center_order,distance,age,rfm
0,0,867689,12,iOS,business,no surge,1,1,1,1,0,0,7.982135,20,low
1,1,752172,5,Android,economy,no surge,1,1,1,1,0,1,2.908468,27,high
2,2,486559,15,Android,comfort,no surge,1,1,1,1,0,0,7.224614,21,high
3,3,304024,0,Android,economy,no surge,1,1,1,1,0,1,1.874349,52,low
4,4,139420,0,Android,business,no surge,1,1,1,1,0,0,10.704778,19,low


In [3]:
import hashlib

def hash_user_id(uid):
    uid_str = str(uid)
    uid_hash = hashlib.md5(uid_str.encode()).hexdigest()  # вернёт hex-строку
    return int(uid_hash, 16)  # переводим hex в int

df["user_hash"] = df["user_id"].apply(hash_user_id)

In [4]:
df["group_index"] = df["user_hash"] % 100


In [5]:

def assign_group(index):
    return "A" if index < 50 else "B"

df["group"] = df["group_index"].apply(assign_group)

In [6]:
print(df["group"].value_counts(normalize=True))


group
B    0.50202
A    0.49798
Name: proportion, dtype: float64


In [7]:
group_counts = df["group"].value_counts()
print(group_counts)

group
B    50955
A    50545
Name: count, dtype: int64


In [8]:
A_count = group_counts.get("A", 0)
B_count = group_counts.get("B", 0)

absolute_diff = abs(A_count - B_count)
relative_diff = absolute_diff / ((A_count + B_count) / 2) * 100

print(f"Абсолютное отклонение: {absolute_diff}")
print(f"Относительное отклонение: {relative_diff:.2f}%")

Абсолютное отклонение: 410
Относительное отклонение: 0.81%


In [9]:
if relative_diff <= 5:
    print("Идеально: разница между группами не превышает 5%. Рандомизация качественная.")
elif relative_diff <= 10:
    print("Допустимо: разница до 10%. Можно использовать, но стоит быть внимательным.")
else:
    print("Критично: разница превышает 10%. Рандомизация требует доработки.")

Идеально: разница между группами не превышает 5%. Рандомизация качественная.


In [13]:
print("\nСравнение возраста:")
print(df.groupby("rfm")["age"].agg(["mean", "median", "std"]))

print("\nРаспределение по операционной системе:")
print(pd.crosstab(df["rfm"], df["os"], normalize="index") * 100)

print("\nРаспределение по заказу в центре города:")
print(pd.crosstab(df["rfm"], df["city_center_order"], normalize="index") * 100)



Сравнение возраста:
             mean  median        std
rfm                                 
high    24.946728    23.0   6.852324
low     26.369518    24.0   8.134743
medium  31.700816    28.0  11.446606

Распределение по операционной системе:
os        Android        iOS
rfm                         
high    55.761948  44.238052
low     55.916409  44.083591
medium  56.281275  43.718725

Распределение по заказу в центре города:
city_center_order          0          1
rfm                                    
high               42.845960  57.154040
low                42.884051  57.115949
medium             41.980511  58.019489
